In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory of src to the path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.db.manager import DBManager
from src.input_to_instructions.load_and_execute import *
from src.input_to_instructions.types import *
from src.operation.execute import *
from src.response_generation.load_and_execute import *
from src.dateutils import normalize_sql_dates


INFO:datasets:PyTorch version 2.7.1+cu128 available.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
from collections import defaultdict
import logging

import pandas as pd
import numpy as np
from tqdm import tqdm
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import json
import itertools

# from db.manager import DBManager
from operation.execute import OperationExecutor
from pathlib import Path
import warnings
import datetime


warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

In [4]:
BASE_DIR = "../"
def read_json(path):
    with open(path, "r", encoding="utf-8") as f:
        result = json.loads(f.read())
    
    # result = [{"Input": d["Input"], "Response": json.dumps(d["Response"], ensure_ascii=False)} for d in result]
    return result

In [5]:
if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
print(f"attn_implementation: {attn_implementation}, torch_dtype: {torch_dtype}")


attn_implementation: flash_attention_2, torch_dtype: torch.bfloat16


# Load data

In [6]:
# BASE_DATASET_DIR = Path("../dataset/v5-250228-multimetadata")
# dataset_name = "v6-250306-optimizetoken"
dataset_name = "v7-250309-reduceinputanddatefunctioncall"
BASE_DATASET_DIR = Path(f"../finetuning/dataset/{dataset_name}")
print(f"BASE_DATASET_DIR: {BASE_DATASET_DIR}")
print(list(BASE_DATASET_DIR.iterdir()))

import re
from datasets import load_dataset, Dataset
train_type = "ours"
def sub(name, common_prompt):
    # Remove the section between <|name|> ... <|name|> including the tags themselves
    # Use re.DOTALL to match newlines with '.'
    pattern = rf"\n?<\|{name}\|>[\s\S]*?<\|{name}\|>"
    common_prompt = re.sub(pattern, "", common_prompt, flags=re.DOTALL)
    return common_prompt

common_prompt = open(BASE_DATASET_DIR / f"prompt.txt", "r").read()

sub_targets = []
if train_type == "ours":
    sub_targets = []
elif train_type == "BASE":
    sub_targets = ["Thinking", "Expectation", "Mapping", "Script", "Examples"]
elif train_type in ["WoThinking"]:
    sub_targets = ["Thinking"]
elif train_type in ["woMetadata"]:
    sub_targets = ["Metadata"]
elif train_type in ["WoMetadata+Thinking"]:
    sub_targets = ["Metadata", "Thinking"]
elif train_type in ["woExp"]:
    sub_targets = ["Expectation"]

if train_type in ["woQM", "woQM+Script"]:
    sub_targets = ["QM", "Mapping"]
if train_type in ["woScript", "woQM+Script"]:
    sub_targets = ["Script"]


for sub_target in sub_targets:
    common_prompt = sub(sub_target, common_prompt)
# remove all <||>
common_prompt = re.sub(r"<\|.*?\|>", "", common_prompt)
print(common_prompt)

scenario_dirs = [d for d in BASE_DATASET_DIR.iterdir() if d.is_dir() and "scenario" in d.name and "metadata.json" in [f.name for f in d.iterdir()]]
print(scenario_dirs)

from src.dateutils import normalize_sql_dates

def read_dataset(dir, path):
    # the file is originally json-list format
    # we want every first-level elements to be a string itself
    # for example, [{"Hi": "a'b'"}, {"Hi": "c'd'"}] -> ["""{"Hi": "a'b'"}""", """{"Hi": "c'd'"}"""]
    
    metadata = json.load(open(dir / "metadata.json", "r"))

    path = dir / path
    with open(path, "r", encoding="utf-8") as f:
        data = json.loads(f.read())
    
    result = []
    for d in data:
        if train_type in ["WoThinking", "WoMetadata+Thinking"]:
            del d["Response"]["Thinking"]
        elif train_type in ["woExp"]:
            del d["Response"]["Expectations"]
        
        if "Script" in d["Response"]:
            if train_type in ["woScript", "woQM+Script"]:
                new_scripts = []
                for i, script in enumerate(d["Response"]["Script"]):
                    if "data" in script:
                        new_scripts.append(script)
                d["Response"]["Script"] = new_scripts

            if train_type in ["woQM"]:
                new_scripts = []
                for i, script in enumerate(d["Response"]["Script"]):
                    if "data" in script:
                        new_scripts.append(script)
                    else:
                        for m, n in [("실내온도", "roomtemp"), ("설정온도", "settemp")]:
                            script = script.replace(f"'{m}'", f"'{n}'")
                        
                        new_scripts.append(script)
                d["Response"]["Script"] = new_scripts

            if train_type in ["woQM", "woQM+Script"]:
                mapping = d["Response"]["Mapping"]
                for i, script in enumerate(d["Response"]["Script"]):
                    if "data" not in script:
                        continue

                    t_match = re.search(r"t=('[^']+'|\[[^\]]+\])", script)
                    s_match = re.search(r"s=('[^']+'|\[[^\]]+\])", script)
                    m_match = re.search(r"m=('[^']+'|\[[^\]]+\])", script)
                    t = eval(t_match.group(1)) if t_match else None
                    s = eval(s_match.group(1)) if s_match else None
                    m = eval(m_match.group(1)) if m_match else None
                    
                    if isinstance(t, str):
                        t = [t]
                    if isinstance(s, str):
                        s = [s]
                    if isinstance(m, str):
                        m = [m]

                    t_raw = [mapping['temporal'][t_highlevel] for t_highlevel in t]
                    s_raw = [mapping['spatials'][s_highlevel] for s_highlevel in s]
                    m_raw = [mapping['modalities'][m_highlevel] for m_highlevel in m]
                    s_raw = [item for sublist in s_raw for item in (sublist if isinstance(sublist, list) else [sublist])]
                    sql = DBManager.get_query_strings_v2(
                        metadata, m_raw, t_raw, s_raw
                    )
                    sql = normalize_sql_dates(sql)
                    # replace data(...) with sql using regex
                    d["Response"]["Script"][i] = re.sub(r"data\(([^)]+)\)", lambda x: f"\"{sql}\"", script)
                del d["Response"]["Mapping"]
            #     # raise NotImplementedError
            # elif train_type in ["woOp"]:
            #     instructions = d["Response"]["Instructions"]
            #     d["Response"]["Instructions"] = [i for i in instructions if i["type"] == "q"]

        tags = d["Tags"]["Style"]

        skip_tags = ["Reason", "Graph", "Unrelated", "Prediction"]

        skip = False
        for skip_tag in skip_tags:
            if skip_tag in tags:
                skip = True
                break
        
        if skip:
            continue

        
        result.append({"Metadata": metadata, "Input": d["Input"], "Scenarios": dir.name, "Response": json.dumps(d["Response"], ensure_ascii=False)})
    # result = [{"Input": d["Input"], "Response": json.dumps(d["Response"], ensure_ascii=False)} for d in data]
    # print(f"Read {len(result)} examples from {path}")
    # print(f"Type of result: {type(result)}")
    # print(f"Type of result[0]: {type(result[0])}")
    # print(f"Type of result[0]['Input']: {type(result[0]['Input'])}")
    # print(f"Type of result[0]['Response']: {type(result[0]['Response'])}")
    return result


dataset_tss = []
for scenario_dir in scenario_dirs:
    dataset_tss.extend(read_dataset(scenario_dir, "onlyq_ts.json"))
    # print("Warning!!!: Test set is not mutually exclusive with training set.")

dataset_ts_ = Dataset.from_list(dataset_tss) # Mutually exclusive한 애들은 None 됨

# print(common_prompt)

BASE_DATASET_DIR: ../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall
[PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/prompt.txt'), PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/refine.ipynb'), PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/scenario1'), PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/scenario2'), PosixPath('../finetuning/dataset/v7-250309-reduceinputanddatefunctioncall/summary.ipynb')]
너는 유저의 HVAC 관련 질문에 답변하는 Agent의 계획을 설계하는 정확하고 훌룡한 인공지능이다. 
사용자의 질문(Input)을 받아 Agent의 Instructions를 출력해 주어야 한다.
Thinking에서는 HVAC 상식을 바탕으로 유저의 의도를 추측해 빠진 context를 추측하여 질문의 모호함을 없앤 완벽한 형태의 질문을 출력하고, 이에 대한 답변 계획을 세워야함.
Expectation에서는 유저가 기대할만한 답변을 추측해야함.
Mapping에서는 질문에 사용된 high-level taxonomy를 metadata를 바탕으로 low-level taxonomy로의 mapping을 계산하여야 함.
이때 metadata에 없는 정보를 기술하는 등 거짓된 출력을 하면 안되고, 모르는 정보가 있으면 Unknown이라 답변해야함.
Script에서는 data 함수를 이용하여 pandas dataframe 형식의 데이터를 쿼리하여 답변에 

# Load export LLM

In [7]:
# model_ids = [
#     '3rdyear_r211_a422_sh2orc-Llama-3.1-Korean-8B-Instruct_tr27',
#     # 'Bllossom/llama-3.2-Korean-Bllossom-AICA-5B',
#     # 'Bllossom/llama-3.2-Korean-Bllossom-3B'
# ]

bit = [4, 8, 16][2]

model_id = "sh2orc-Llama-3.1-Korean-8B-Instruct"
model_repr = '3rdyear_r211_a422_sh2orc-Llama-3.1-Korean-8B-Instruct_tr27'
checkpoint_num = 41

# model_id = "Bllossom-llama-3.2-Korean-Bllossom-3B"
# model_repr = '3rdyear_r450_a900_Bllossom-llama-3.2-Korean-Bllossom-3B_tr27'
# checkpoint_num = 70


expertLLM_dir = Path("/model") / model_id
checkpoint_dir = expertLLM_dir / f"chkpts/{model_repr}/checkpoint-{checkpoint_num}"
cache_dir = expertLLM_dir / "cache"

In [8]:
import pynvml
import threading
import time

class GPUMonitor:
    def __init__(self):
        self.max_memory = 0
        self.start_memory = 0
        self.running = False
        self.thread = None
        pynvml.nvmlInit()
        self.handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    
    def get_memory(self):
        mem_info = pynvml.nvmlDeviceGetMemoryInfo(self.handle)
        used_gb = mem_info.used / 1024**3
        return used_gb

    def _monitor(self):
        while self.running:
            used_gb = self.get_memory()
            self.max_memory = max(self.max_memory, used_gb)
            time.sleep(0.5)
    
    def start(self, start_memory=None):
        # 시작 시점 메모리 측정
        if start_memory is not None:
            self.start_memory = start_memory
        else:
            self.start_memory = self.get_memory()
        self.max_memory = self.start_memory
        
        self.running = True
        self.thread = threading.Thread(target=self._monitor)
        self.thread.start()
    
    def stop(self):
        self.running = False
        if self.thread:
            self.thread.join()
        # 최대값에서 시작값을 뺀 증가량 반환
        return round(self.max_memory - self.start_memory, 2)
    
    def __del__(self):
        pynvml.nvmlShutdown()

# 사용 예시
monitor = GPUMonitor()


In [ ]:
start_memory = monitor.get_memory()
# 모델 및 토크나이저 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    checkpoint_dir.as_posix(),
    attn_implementation=attn_implementation,
    dtype=torch_dtype,
    max_seq_length=2048,
    load_in_4bit=True if bit == 4 else False,
    load_in_8bit=True if bit == 8 else False,
    cache_dir=cache_dir.as_posix(),
    local_files_only=True,
    device_map="cuda",
)
tokenizer.padding_side = "left"

def formatting_prompts_func(examples):
    convos = []
    inputs = []
    for metadata, input, response in zip(examples['Metadata'], examples['Input'], examples['Response']):
        response.replace("    ", "")
        answer = {
            "content": f"{response}",
            "role": "assistant"
        }
        prompt = {
            "content": common_prompt,
            "role": "system"
        }
        content = ""
        if train_type not in ["WoMetadata", "WoMetadata+Thinking"]:
            content += f"Metadata:{metadata};"
        content += f"Input:{input};"
        user_input = {
            "content": content,
            "role": "user"
        }
        convos.append([prompt, user_input, answer])
        inputs.append([prompt, user_input])
    
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos]
    inputs = [
        tokenizer.apply_chat_template(input, tokenize=False, add_generation_prompt=True)
        for input in inputs
    ]
    return {"text": texts, "input": inputs}

dataset_ts = dataset_ts_.map(formatting_prompts_func, batched=True)

eval_inputs = dataset_ts['input']
inputs = dataset_ts['Input']
scenarios = dataset_ts['Scenarios']
metadatas = dataset_ts['Metadata']

# 토큰화 (패딩 없이)
tokenized_inputs = []
for text_input in eval_inputs:
    tokens = tokenizer(
        text_input,
        return_tensors="pt",
        padding=False,
        truncation=True,
        max_length=2048
    )['input_ids'].to(model.device)
    tokenized_inputs.append(tokens)

mdscs = [(md, sc) for md, sc in zip(metadatas, scenarios)]

# 추론 준비
monitor.start(start_memory=start_memory)
FastLanguageModel.for_inference(model)
# 개별 추론 및 측정
results = []
time_records = []
memory_records = []

print(f"Starting individual inference for {len(tokenized_inputs)} samples...")
print("-" * 60)

for repeat_idx in range(2):
    print(f"Repeat {repeat_idx + 1}/3")
    for idx, input_tensor in enumerate(tokenized_inputs):
        print(f"Sample {idx+1}/{len(tokenized_inputs)}")
        
        # 메모리 모니터링 시작
        
        # 시간 측정 시작
        start_time = time.time()
        
        with torch.no_grad():
            # 개별 샘플 추론
            output = model.generate(
                input_ids=input_tensor,
                max_new_tokens=2000,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
            )
        
        # 시간 측정 종료
        inference_time = time.time() - start_time
        
        
        # 디코딩
        decoded = tokenizer.decode(output[0], skip_special_tokens=False)
        
        # 결과 저장
        try:
            out = extract_content(decoded)
            out = eval(out)
        except:
            out = decoded
        
        result = {
            "Input": inputs[idx],
            "Scenario": mdscs[idx][1],
            "Metadata": mdscs[idx][0],
            "Candidate": out
        }
        results.append(result)
        
        # 측정값 저장
        time_records.append(inference_time)
        
        # 진행상황 출력
        # print(f"  - Time: {inference_time:.2f}s")
        # print(f"  - Input length: {input_tensor.size(1)} tokens")
        # print()


# 메모리 측정 종료
peak_memory = monitor.stop()

# 통계 출력

# print(f"Total time: {sum(time_records):.2f}s")

# # 결과 저장
# save_dir = "../experiments/result_3rdyear"
# output_path = os.path.join(save_dir, f"r-{model_repr}_{bit}bit-step-{checkpoint_num}.json")

# with open(output_path, "w", encoding="utf-8") as f:
#     json_str = json.dumps(results, ensure_ascii=False, indent=2)
#     f.write(json_str)

# print(f"\nResults saved to: {output_path}")

==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.19 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Unsloth 2025.8.1 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Starting individual inference for 12 samples...
------------------------------------------------------------
Repeat 1/3
Sample 1/12
Sample 2/12
Sample 3/12
Sample 4/12


KeyboardInterrupt: 

In [ ]:
print(f"r-{model_repr}_{bit}bit-step-{checkpoint_num}")
print("=" * 60)
print(f"peak memory: {peak_memory} GB")

print(f"Inference time avg, min, max, std: {np.mean(time_records):.2f}s, {np.min(time_records):.2f}s, {np.max(time_records):.2f}s, {np.std(time_records):.2f}s")

# 각 샘플별 표준편차 계산 (여러 번 실행한 경우)
num_unique_samples = len(tokenized_inputs)
if len(time_records) > num_unique_samples:  # 반복 실행한 경우
    # 각 샘플별로 시간 기록 그룹화
    sample_times = {i: [] for i in range(num_unique_samples)}
    for idx, time_val in enumerate(time_records):
        sample_idx = idx % num_unique_samples
        sample_times[sample_idx].append(time_val)
    
    # 각 샘플별 표준편차 계산
    sample_stds = []
    for sample_idx in range(num_unique_samples):
        if len(sample_times[sample_idx]) > 1:
            sample_std = np.std(sample_times[sample_idx])
            sample_stds.append(sample_std)
    if sample_stds:
        print(f"Average per-sample std: {np.mean(sample_stds):.3f}s")


r-3rdyear_r450_a900_Bllossom-llama-3.2-Korean-Bllossom-3B_tr27_16bit-step-70
peak memory: 12.08 GB
Inference time avg, min, max, std: 26.19s, 24.81s, 26.46s, 0.26s
Average per-sample std: 0.139s


r-3rdyear_r211_a422_sh2orc-Llama-3.1-Korean-8B-Instruct_tr27_4bit-step-41
============================================================
peak memory: 10.22 GB
Inference time avg, min, max, std: 5.72s, 1.69s, 15.12s, 3.38s
Average per-sample std: 0.137s

r-3rdyear_r211_a422_sh2orc-Llama-3.1-Korean-8B-Instruct_tr27_8bit-step-41
============================================================
peak memory: 17.9 GB
Inference time avg, min, max, std: 9.57s, 2.93s, 17.45s, 4.11s
Average per-sample std: 0.580s

r-3rdyear_r211_a422_sh2orc-Llama-3.1-Korean-8B-Instruct_tr27_16bit-step-41
============================================================
peak memory: 19.86 GB
Inference time avg, min, max, std: 5.25s, 1.66s, 11.44s, 2.40s
Average per-sample std: 0.273s

r-3rdyear_r450_a900_Bllossom-llama-3.2-Korean-Bllossom-3B_tr27_16bit-step-70
============================================================
peak memory: 12.08 GB
Inference time avg, min, max, std: 26.19s, 24.81s, 26.46s, 0.26s
Average per-sample std: 0.139s